In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Import the test data in the Google Drive

In [0]:
import os
import pandas as pd
import numpy as np

root = os.getcwd()
path = os.path.join(root, 'gdrive/My Drive/Deepimpute/test.csv')

# 1. Loading the test data  

. I am analyzing the **Deepimpute** software (Arisdakessian *et al*)  
. This software is implemented in a class called **MultiNet**  
(https://github.com/lanagarmire/DeepImpute) 

In [0]:
raw = pd.read_csv(path, index_col = 0) # (500, 3000)
raw.head()

,ENSG00000177954,ENSG00000197756,ENSG00000231500,ENSG00000140988,ENSG00000105372,ENSG00000198712,ENSG00000109475,ENSG00000112306,ENSG00000137818,ENSG00000115268,ENSG00000108298,ENSG00000142541,ENSG00000186468,ENSG00000164587,ENSG00000198899,ENSG00000149273,ENSG00000177600,ENSG00000144713,ENSG00000171863,ENSG00000167526,ENSG00000108107,ENSG00000251562,ENSG00000130255,ENSG00000105193,ENSG00000197061,ENSG00000137154,ENSG00000166441,ENSG00000142676,ENSG00000187514,ENSG00000083845,ENSG00000136942,ENSG00000167996,ENSG00000134419,ENSG00000205542,ENSG00000145592,ENSG00000198886,ENSG00000143947,ENSG00000125691,ENSG00000149806,ENSG00000198763,...,ENSG00000153485,ENSG00000244045,ENSG00000119718,ENSG00000177508,ENSG00000178307,ENSG00000142444,ENSG00000169180,ENSG00000100116,ENSG00000108590,ENSG00000025293,ENSG00000101266,ENSG00000072849,ENSG00000132581,ENSG00000247092,ENSG00000109111,ENSG00000102900,ENSG00000126787,ENSG00000260708,ENSG00000071564,ENSG00000153944,ENSG00000132612,ENSG00000108639,ENSG00000213015,ENSG00000167900,ENSG00000119559,ENSG00000197696,ENSG00000160209,ENSG00000105698,ENSG00000159111,ENSG00000125375,ENSG00000269858,ENSG00000182087,ENSG00000160214,ENSG00000166411,ENSG00000186153,ENSG00000089351,ENSG00000108433,ENSG00000206053,ENSG00000137806,ENSG00000197766
AATTGTGACTACGA-1,826.0,674.0,694.0,809.0,771.0,796.0,755.0,684.0,597.0,516.0,622.0,493.0,492.0,490.0,561.0,446.0,555.0,511.0,494.0,510.0,451.0,259.0,471.0,471.0,224.0,520.0,451.0,456.0,384.0,407.0,422.0,356.0,418.0,265.0,406.0,441.0,367.0,330.0,352.0,447.0,...,1.0,0.0,4.0,0.0,2.0,2.0,8.0,5.0,5.0,0.0,2.0,5.0,3.0,4.0,1.0,3.0,1.0,3.0,3.0,3.0,0.0,3.0,4.0,1.0,2.0,2.0,1.0,2.0,1.0,3.0,3.0,2.0,3.0,2.0,2.0,0.0,0.0,1.0,5.0,4.0
TGACACGATTCGTT-1,617.0,618.0,594.0,703.0,671.0,473.0,549.0,523.0,476.0,527.0,520.0,449.0,433.0,407.0,419.0,421.0,467.0,429.0,377.0,439.0,384.0,389.0,372.0,440.0,384.0,415.0,366.0,362.0,368.0,353.0,377.0,205.0,320.0,304.0,320.0,349.0,335.0,247.0,307.0,274.0,...,2.0,1.0,3.0,2.0,2.0,4.0,1.0,1.0,2.0,2.0,3.0,3.0,4.0,1.0,4.0,3.0,1.0,0.0,1.0,5.0,4.0,3.0,1.0,1.0,3.0,1.0,1.0,1.0,3.0,5.0,2.0,1.0,2.0,2.0,3.0,3.0,1.0,4.0,2.0,3.0
TGTCAGGATTGTCT-1,525.0,550.0,540.0,546.0,615.0,565.0,263.0,422.0,449.0,518.0,451.0,364.0,402.0,482.0,414.0,584.0,429.0,334.0,359.0,360.0,359.0,89.0,314.0,329.0,14.0,279.0,394.0,322.0,221.0,313.0,240.0,141.0,237.0,524.0,237.0,224.0,134.0,258.0,408.0,212.0,...,0.0,1.0,2.0,3.0,3.0,0.0,0.0,0.0,1.0,1.0,0.0,3.0,3.0,2.0,2.0,3.0,3.0,4.0,4.0,1.0,1.0,3.0,1.0,2.0,3.0,2.0,2.0,4.0,3.0,4.0,2.0,1.0,0.0,3.0,1.0,2.0,3.0,2.0,2.0,1.0
TAAGTAACGTTCTT-1,514.0,474.0,361.0,331.0,447.0,279.0,188.0,332.0,379.0,380.0,271.0,233.0,322.0,387.0,215.0,386.0,390.0,317.0,257.0,243.0,275.0,85.0,205.0,252.0,258.0,234.0,270.0,250.0,279.0,251.0,256.0,122.0,113.0,269.0,266.0,140.0,89.0,128.0,303.0,229.0,...,0.0,0.0,1.0,0.0,3.0,2.0,1.0,1.0,0.0,3.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,1.0,3.0,0.0,0.0,1.0,1.0,2.0,0.0,3.0,3.0,0.0,1.0,0.0,3.0,1.0
TCCTAAACTTCATC-1,444.0,507.0,509.0,566.0,520.0,246.0,352.0,413.0,381.0,481.0,369.0,309.0,361.0,412.0,205.0,338.0,366.0,387.0,337.0,294.0,261.0,96.0,350.0,344.0,106.0,281.0,275.0,311.0,270.0,248.0,327.0,145.0,269.0,166.0,266.0,155.0,164.0,217.0,278.0,100.0,...,2.0,0.0,0.0,0.0,2.0,3.0,2.0,3.0,3.0,1.0,1.0,2.0,2.0,0.0,0.0,3.0,0.0,3.0,1.0,2.0,4.0,1.0,2.0,3.0,1.0,3.0,1.0,3.0,0.0,2.0,1.0,2.0,0.0,5.0,4.0,0.0,3.0,4.0,2.0,1.0


In [0]:
vmr = raw.std() / raw.mean() # colwise calculation
vmr[np.isinf(vmr)] = 0

In [0]:
potential_pred = raw.columns[vmr > 0]
potential_pred # cell names

Calculate a covariance matrix (cells x cells)  
This will be used for selecting output cells when building a NN model

In [0]:
covariance_matrix = pd.DataFrame(np.abs(np.corrcoef(raw.T.loc[potential_pred])),
                                 index = potential_pred,
                                 columns = potential_pred).fillna(0)

In [0]:
covariance_matrix.head()

,ENSG00000177954,ENSG00000197756,ENSG00000231500,ENSG00000140988,ENSG00000105372,ENSG00000198712,ENSG00000109475,ENSG00000112306,ENSG00000137818,ENSG00000115268,ENSG00000108298,ENSG00000142541,ENSG00000186468,ENSG00000164587,ENSG00000198899,ENSG00000149273,ENSG00000177600,ENSG00000144713,ENSG00000171863,ENSG00000167526,ENSG00000108107,ENSG00000251562,ENSG00000130255,ENSG00000105193,ENSG00000197061,ENSG00000137154,ENSG00000166441,ENSG00000142676,ENSG00000187514,ENSG00000083845,ENSG00000136942,ENSG00000167996,ENSG00000134419,ENSG00000205542,ENSG00000145592,ENSG00000198886,ENSG00000143947,ENSG00000125691,ENSG00000149806,ENSG00000198763,...,ENSG00000153485,ENSG00000244045,ENSG00000119718,ENSG00000177508,ENSG00000178307,ENSG00000142444,ENSG00000169180,ENSG00000100116,ENSG00000108590,ENSG00000025293,ENSG00000101266,ENSG00000072849,ENSG00000132581,ENSG00000247092,ENSG00000109111,ENSG00000102900,ENSG00000126787,ENSG00000260708,ENSG00000071564,ENSG00000153944,ENSG00000132612,ENSG00000108639,ENSG00000213015,ENSG00000167900,ENSG00000119559,ENSG00000197696,ENSG00000160209,ENSG00000105698,ENSG00000159111,ENSG00000125375,ENSG00000269858,ENSG00000182087,ENSG00000160214,ENSG00000166411,ENSG00000186153,ENSG00000089351,ENSG00000108433,ENSG00000206053,ENSG00000137806,ENSG00000197766
ENSG00000177954,1.000000,0.765914,0.795157,0.715283,0.855736,0.646341,0.653232,0.857135,0.812389,0.811844,0.812909,0.818119,0.827483,0.784971,0.706441,0.802682,0.827017,0.763311,0.720066,0.707946,0.858446,0.126456,0.839807,0.805154,0.072539,0.776600,0.857326,0.791923,0.477497,0.762747,0.623310,0.254908,0.844566,0.476813,0.728109,0.647625,0.603197,0.831229,0.706061,0.557215,...,0.087396,0.066376,0.065912,0.159194,0.094218,0.040567,0.314734,0.065176,0.159631,0.060454,0.074380,0.139784,0.130224,0.136853,0.166262,0.099585,0.020821,0.175572,0.119465,0.195646,0.111455,0.247940,0.214076,0.117249,0.150756,0.224172,0.057516,0.192885,0.069353,0.195561,0.105862,0.102781,0.040017,0.023280,0.079441,0.090609,0.048090,0.146617,0.060840,0.177601
ENSG00000197756,0.765914,1.000000,0.823399,0.697642,0.757539,0.383251,0.432227,0.774014,0.701896,0.799065,0.750441,0.737588,0.706696,0.702288,0.514315,0.766292,0.780877,0.759288,0.845848,0.712982,0.749665,0.079521,0.813701,0.773451,0.174302,0.753418,0.793746,0.762458,0.684528,0.652877,0.723583,0.250306,0.771539,0.493503,0.720498,0.382317,0.464397,0.772276,0.712628,0.478716,...,0.128090,0.101431,0.033157,0.122738,0.078991,0.096732,0.297153,0.050941,0.068576,0.109529,0.100698,0.108903,0.175999,0.087436,0.177659,0.108607,0.009037,0.132224,0.089365,0.164839,0.112660,0.199997,0.105470,0.121079,0.136657,0.175845,0.023836,0.120133,0.082735,0.192801,0.108089,0.104475,0.089434,0.090052,0.153040,0.135835,0.110299,0.181901,0.051941,0.136466
ENSG00000231500,0.795157,0.823399,1.000000,0.808072,0.816690,0.349218,0.414345,0.853762,0.790331,0.834374,0.829422,0.834121,0.792027,0.828762,0.478897,0.838756,0.854875,0.846182,0.797916,0.791570,0.810255,0.107676,0.841065,0.810901,0.076118,0.793828,0.835751,0.836875,0.523724,0.742111,0.723833,0.294563,0.829890,0.460747,0.781120,0.373974,0.368374,0.795283,0.743056,0.452141,...,0.132733,0.071588,0.073324,0.084023,0.078860,0.093319,0.294085,0.093499,0.075230,0.063699,0.084471,0.152162,0.154608,0.085922,0.181641,0.102155,0.014883,0.143915,0.120503,0.161426,0.144523,0.211397,0.181933,0.084706,0.159287,0.197940,0.031398,0.184126,0.083043,0.174690,0.129217,0.118452,0.116049,0.075341,0.103637,0.139540,0.073267,0.161118,0.101250,0.131862
ENSG00000140988,0.715283,0.697642,0.808072,1.000000,0.808984,0.332613,0.335548,0.821013,0.762533,0.791263,0.830228,0.794995,0.675774,0.709829,0.436097,0.793805,0.817407,0.779602,0.716445,0.785425,0.781074,0.044339,0.772640,0.803073,0.078028,0.731963,0.776273,0.794402,0.473209,0.754978,0.685402,0.307409,0.796404,0.427304,0.677673,0.364924,0.299198,0.677230,0.723127,0.453578,...,0.109441,0.085630,0.097597,0.093149,0.135089,0.115402,0.310024,0.088669,0.132162,0.111050,0.070555,0.229060,0.18838

In [0]:
gene_metric = (raw.var() / (1 + raw.mean())).sort_values(ascending = False)
gene_metric = gene_metric[gene_metric > 0] # list of genes and its VMRs
gene_metric

ENSG00000197061    89.231862
ENSG00000109475    72.205108
ENSG00000198712    71.810934
ENSG00000197756    37.314497
ENSG00000177954    37.111399
                     ...    
ENSG00000175376     0.450955
ENSG00000147526     0.450052
ENSG00000141385     0.443927
ENSG00000090061     0.443431
ENSG00000169592     0.429001
Length: 3000, dtype: float64

In [0]:
minVMR = 0.5 # Minimum VMR to use
NN_lim = (gene_metric > minVMR).sum()
NN_lim

2938

In [0]:
n_subsets = int(np.ceil(NN_lim / 512)) # 6 subsets
genes_to_impute = gene_metric.index[:n_subsets*512] # 3000 genes
genes_to_impute

Index(['ENSG00000197061', 'ENSG00000109475', 'ENSG00000198712',
       'ENSG00000197756', 'ENSG00000177954', 'ENSG00000138326',
       'ENSG00000167996', 'ENSG00000143947', 'ENSG00000161970',
       'ENSG00000231500',
       ...
       'ENSG00000147471', 'ENSG00000107679', 'ENSG00000255135',
       'ENSG00000105671', 'ENSG00000156110', 'ENSG00000175376',
       'ENSG00000147526', 'ENSG00000141385', 'ENSG00000090061',
       'ENSG00000169592'],
      dtype='object', length=3000)

In [0]:
rest = 512 - (len(genes_to_impute) % 512) # output sub-dimension = 512
rest

72

In [0]:
fill_genes = np.random.choice(gene_metric.index, rest)
# Select 72 genes randomely

In [0]:
genes_to_impute = np.concatenate([genes_to_impute, fill_genes]) 
# So the number of genes to impute are 3072, this means we can evenly divide the genes into 6 subsets having exactly 512 genes each

In [0]:
3072 / 512

6.0

In [0]:
# setTarget(raw.reindex(columns = gene_to_impute), mode = mode) mode = random
# setPredictirs(covariance_matrix, ntop = ntop) ntop = 5
#raw.reindex(columns = genes_to_impute) # 500 * 3072

data = raw.reindex(columns = genes_to_impute)
targets = np.random.choice(data.columns, 
                           [n_subsets, 512],
                           replace = False) # randomely choose 6 subsets with 512 each without replacement

targets_a = targets # copying the object because the object 'target' will be modified by the below loop.

In [0]:
targets.shape

(6, 512)

In [0]:
all_predictors = []
for i, targets in enumerate(targets):
  subMatrix = ( covariance_matrix
               .loc[targets]
               .drop(np.intersect1d(targets, covariance_matrix.columns), axis = 1) # unique rows and columns
               )
  sorted_idx = np.argsort(-subMatrix.values, axis = 1) # sort rowwise, returns indices
  predictors = subMatrix.columns[sorted_idx[:,:5].flatten()] # only get highly correlated cells and their values

  all_predictors.append(predictors.unique())

  print("Net {}: {} predictors, {} targets"
       .format(i, len(np.unique(predictors)), len(targets)))
  
# predictors genes are not same as the target genes but highly correlated
# predictor genes are used as input genes to predict the target genes, so they are 'imputed' along the training procedure

Net 0: 596 predictors, 512 targets
Net 1: 587 predictors, 512 targets
Net 2: 596 predictors, 512 targets
Net 3: 616 predictors, 512 targets
Net 4: 603 predictors, 512 targets
Net 5: 596 predictors, 512 targets


In [0]:
norm_data = np.log1p(raw).astype(np.float32)

# ----- End of Data Prep ----- #

# 2. Training a NN model

In [0]:
# ----- Loading TF and Keras ----- #

import tensorflow.keras as keras
from keras import backend as k
from keras.models import Model
from keras.layers import Dense, Dropout, Input
from keras.callbacks import EarlyStopping
import keras.losses

import tensorflow as tf

Using TensorFlow backend.


Defining a loss function

In [0]:
def wMSE(y_true, y_pred, binary = False): # weighted mean-square error
  if binary:
    weights = tf.cast(y_true > 0, tf.float32)
  else:
    weights = y_true
  
  return tf.reduce_mean(weights*tf.square(y_true-y_pred))

Setting up the parameters

In [0]:
NN_parameters = {"learning_rate": 1e-4, 
                 "batch_size": 64,
                 "loss": wMSE,
                 "architecture": None,
                 "max_epochs": 500,
                 "patience": 5
                 }

NN_parameters['architecture'] = [
        {"type": "dense", "neurons": 256, "activation": "relu"},
        {"type": "dropout", "rate": 0.2},                                 
]

In [0]:
a_seed = 1234

def build(inputdims):

    print(NN_parameters['architecture'])

    inputs = [ Input(shape=(inputdim,)) for inputdim in inputdims ]
    outputs = inputs

    for layer in NN_parameters['architecture']:
        if layer['type'].lower() == 'dense':
            outputs = [ Dense(layer['neurons'], activation=layer['activation'])(output) 
                            for output in outputs ]
        elif layer['type'].lower() == 'dropout':
            outputs = [ Dropout(layer['rate'], seed=a_seed)(output)
                            for output in outputs] 
        else:
            print("Unknown layer type.")

    outputs = [Dense(512, activation="softplus")(output)
              for output in outputs]
      
    model = Model(inputs = inputs, outputs = outputs)

    loss = NN_parameters['loss']

    model.compile(optimizer = keras.optimizers.Adam(lr = NN_parameters['learning_rate']),
                  loss = loss)
    
    return model

Building a Keras model

In [0]:
model = build([len(genes) for genes in all_predictors])

[{'type': 'dense', 'neurons': 256, 'activation': 'relu'}, {'type': 'dropout', 'rate': 0.2}]


In [0]:
model.layers # refer to the Keras API documentation for how these layers are all added. 6 inputs. There are 6 sub-NNs

In [0]:
test_cells = np.random.choice(norm_data.index, int(0.05 * norm_data.shape[0]), replace=False)
train_cells = np.setdiff1d(norm_data.index, test_cells)

In [0]:
X_train = [norm_data.loc[train_cells, inputgenes].values for inputgenes in all_predictors]
Y_train = [norm_data.loc[train_cells, targetgenes].values for targetgenes in targets_a] 

X_test = [norm_data.loc[test_cells, inputgenes].values for inputgenes in all_predictors]
Y_test = [norm_data.loc[test_cells, targetgenes].values for targetgenes in targets_a]

Fitting the data

In [0]:
print("Fitting with {} cells".format(norm_data.shape[0]))

result = model.fit(X_train, Y_train, # This is the Keras method, not the MultiNet method
                   validation_data = (X_test, Y_test),
                   epochs = NN_parameters["max_epochs"],
                   batch_size = NN_parameters["batch_size"],
                   callbacks = [EarlyStopping(monitor = 'val_loss',
                                              patience = NN_parameters["patience"])]
                   )

trained_epochs = len(result.history['loss'])
print("Stopped fitting after {} epochs".format(trained_epochs))

Fitting with 500 cells
Train on 475 samples, validate on 25 samples
Epoch 1/500
475/475 [==============================] - 1s 2ms/step - loss: 28.6571 - dense_7_loss: 5.2371 - dense_8_loss: 4.1549 - dense_9_loss: 4.5273 - dense_10_loss: 4.0511 - dense_11_loss: 4.7154 - dense_12_loss: 5.7111 - val_loss: 23.0146 - val_dense_7_loss: 4.3474 - val_dense_8_loss: 3.2442 - val_dense_9_loss: 3.6218 - val_dense_10_loss: 3.2155 - val_dense_11_loss: 3.9431 - val_dense_12_loss: 4.6426
Epoch 2/500
475/475 [==============================] - 0s 653us/step - loss: 21.9096 - dense_7_loss: 3.9961 - dense_8_loss: 3.0971 - dense_9_loss: 3.4470 - dense_10_loss: 3.0476 - dense_11_loss: 3.6462 - dense_12_loss: 4.4875 - val_loss: 17.0011 - val_dense_7_loss: 3.1790 - val_dense_8_loss: 2.3414 - val_dense_9_loss: 2.7137 - val_dense_10_loss: 2.2871 - val_dense_11_loss: 2.9277 - val_dense_12_loss: 3.5522
Epoch 3/500
475/475 [==============================] - 0s 662us/step - loss: 17.1931 - dense_7_loss: 3.1571 - de

Compute the MSE, note that the predict method here is the one from Keras.  
Not the method of this MultiNet class

In [0]:
Y_test_imputed = np.hstack(model.predict(X_test)).flatten()
Y_test_raw = np.hstack(Y_test).flatten()

Y_test_imputed = Y_test_imputed[Y_test_raw > 0]
Y_test_raw = Y_test_raw[Y_test_raw > 0]

mse = np.sum((Y_test_raw - Y_test_imputed)**2) / len(Y_test_raw)

In [0]:
Y_test_imputed

array([1.1533892, 1.7027059, 1.731009 , ..., 1.7241787, 1.1907964,
       1.1159554], dtype=float32)

In [0]:
Y_test_raw

array([1.0986123, 2.3025851, 1.0986123, ..., 1.7917595, 1.0986123,
       1.0986123], dtype=float32)

In [0]:
mse

0.16926930484905617

Now, we use the built model to impute the raw data

In [0]:
def predict(raw,
            imputed_only=False,
            policy="restore"):
  
  norm_raw = np.log1p(raw)

  inputs = [ norm_raw.loc[:,predictors].values.astype(np.float32)
   for predictors in all_predictors]

  predicted = model.predict(inputs)

  if len(inputs) > 1:
    predicted = np.hstack(predicted) # horizontal stack, cbind() in R

  predicted = pd.DataFrame(predicted, index = raw.index, columns = targets_a.flatten())

  predicted = predicted.groupby(by = predicted.columns, axis = 1).mean() # 72 cells that were filled above are repetitions
  not_predicted = norm_raw.drop(targets_a.flatten(), axis = 1)

  imputed = (pd.concat([predicted, not_predicted], axis = 1)
  .loc[raw.index, raw.columns]
  .values)

  imputed[ (imputed > 2*norm_raw.values.max()) | (np.isnan(imputed)) ] = 0

  imputed = np.expm1(imputed)

  if policy == "restore":
    print("Filling zeros")
    mask = (raw.values > 0)
    imputed[mask] = raw.values[mask]

  imputed = pd.DataFrame(imputed, index = raw.index, columns = raw.columns)

  return imputed    

In [0]:
imputed = predict(raw, imputed_only=False, policy='restore')

Filling zeros


In [0]:
imputed

,ENSG00000177954,ENSG00000197756,ENSG00000231500,ENSG00000140988,ENSG00000105372,ENSG00000198712,ENSG00000109475,ENSG00000112306,ENSG00000137818,ENSG00000115268,ENSG00000108298,ENSG00000142541,ENSG00000186468,ENSG00000164587,ENSG00000198899,ENSG00000149273,ENSG00000177600,ENSG00000144713,ENSG00000171863,ENSG00000167526,ENSG00000108107,ENSG00000251562,ENSG00000130255,ENSG00000105193,ENSG00000197061,ENSG00000137154,ENSG00000166441,ENSG00000142676,ENSG00000187514,ENSG00000083845,ENSG00000136942,ENSG00000167996,ENSG00000134419,ENSG00000205542,ENSG00000145592,ENSG00000198886,ENSG00000143947,ENSG00000125691,ENSG00000149806,ENSG00000198763,...,ENSG00000153485,ENSG00000244045,ENSG00000119718,ENSG00000177508,ENSG00000178307,ENSG00000142444,ENSG00000169180,ENSG00000100116,ENSG00000108590,ENSG00000025293,ENSG00000101266,ENSG00000072849,ENSG00000132581,ENSG00000247092,ENSG00000109111,ENSG00000102900,ENSG00000126787,ENSG00000260708,ENSG00000071564,ENSG00000153944,ENSG00000132612,ENSG00000108639,ENSG00000213015,ENSG00000167900,ENSG00000119559,ENSG00000197696,ENSG00000160209,ENSG00000105698,ENSG00000159111,ENSG00000125375,ENSG00000269858,ENSG00000182087,ENSG00000160214,ENSG00000166411,ENSG00000186153,ENSG00000089351,ENSG00000108433,ENSG00000206053,ENSG00000137806,ENSG00000197766
AATTGTGACTACGA-1,826.0,674.0,694.0,809.0,771.0,796.0,755.0,684.0,597.0,516.0,622.0,493.0,492.0,490.0,561.0,446.0,555.0,511.0,494.0,510.0,451.0,259.0,471.0,471.0,224.0,520.0,451.0,456.0,384.0,407.0,422.0,356.0,418.0,265.0,406.0,441.0,367.0,330.0,352.0,447.0,...,1.000000,1.732275,4.000000,1.594862,2.0,2.000000,8.000000,5.000000,5.000000,2.066799,2.000000,5.000000,3.000000,4.000000,1.000000,3.000000,1.000000,3.000000,3.00000,3.000000,2.172500,3.000000,4.000000,1.000000,2.000000,2.000000,1.000000,2.000000,1.000000,3.000000,3.000000,2.000000,3.000000,2.000000,2.000000,1.022960,1.890383,1.000000,5.000000,4.0
TGACACGATTCGTT-1,617.0,618.0,594.0,703.0,671.0,473.0,549.0,523.0,476.0,527.0,520.0,449.0,433.0,407.0,419.0,421.0,467.0,429.0,377.0,439.0,384.0,389.0,372.0,440.0,384.0,415.0,366.0,362.0,368.0,353.0,377.0,205.0,320.0,304.0,320.0,349.0,335.0,247.0,307.0,274.0,...,2.000000,1.000000,3.000000,2.000000,2.0,4.000000,1.000000,1.000000,2.000000,2.000000,3.000000,3.000000,4.000000,1.000000,4.000000,3.000000,1.000000,1.156944,1.00000,5.000000,4.000000,3.000000,1.000000,1.000000,3.000000,1.000000,1.000000,1.000000,3.000000,5.000000,2.000000,1.000000,2.000000,2.000000,3.000000,3.000000,1.000000,4.000000,2.000000,3.0
TGTCAGGATTGTCT-1,525.0,550.0,540.0,546.0,615.0,565.0,263.0,422.0,449.0,518.0,451.0,364.0,402.0,482.0,414.0,584.0,429.0,334.0,359.0,360.0,359.0,89.0,314.0,329.0,14.0,279.0,394.0,322.0,221.0,313.0,240.0,141.0,237.0,524.0,237.0,224.0,134.0,258.0,408.0,212.0,...,1.369816,1.000000,2.000000,3.000000,3.0,2.174685,3.561759,1.517931,1.000000,1.000000,2.048922,3.000000,3.000000,2.000000,2.000000,3.000000,3.000000,4.000000,4.00000,1.000000,1.000000,3.000000,1.000000,2.000000,3.000000,2.000000,2.000000,4.000000,3.000000,4.000000,2.000000,1.000000,2.275960,3.000000,1.000000,2.000000,3.000000,2.000000,2.000000,1.0
TAAGTAACGTTCTT-1,514.0,474.0,361.0,331.0,447.0,279.0,188.0,332.0,379.0,380.0,271.0,233.0,322.0,387.0,215.0,386.0,390.0,317.0,257.0,243.0,275.0,85.0,205.0,252.0,258.0,234.0,270.0,250.0,279.0,251.0,256.0,122.0,113.0,269.0,266.0,140.0,89.0,128.0,303.0,229.0,...,1.696681,1.799844,1.000000,1.936108,3.0,2.000000,1.000000,1.000000,1.506088,3.000000,2.227704,1.457231,1.619649,1.316617,1.000000,1.000000,1.000000,1.466089,1.33469,1.550839,4.000000,1.588555,1.288942,1.336698,1.379673,1.000000,3.000000,1.716497,3.069471,1.000000,1.000000,2.000000,2.335917,3.000000,3.000000,1.062540,1.000000,1.334777,3.000000,1.0
TCCTAAACTTCATC-1,444.0,507.0,509.0,566.0,520.0,246.0,352.0,413.0,381.0,481.0,369.0,309.0,361.0,412.0,205.0,338.0,366.0,387.0,337.0,294.0,261.0,96.0,350.0,344.0,106.0,281.0,275.0,311.0,270.0,248.0,327.0,145.0,269.0,166.0,266.0,155.0,164.0,217.0,278.0,100.0,...,2.000000,1.851829,1

Note the genes that were previously 0s are now imputed

In [0]:
raw

,ENSG00000177954,ENSG00000197756,ENSG00000231500,ENSG00000140988,ENSG00000105372,ENSG00000198712,ENSG00000109475,ENSG00000112306,ENSG00000137818,ENSG00000115268,ENSG00000108298,ENSG00000142541,ENSG00000186468,ENSG00000164587,ENSG00000198899,ENSG00000149273,ENSG00000177600,ENSG00000144713,ENSG00000171863,ENSG00000167526,ENSG00000108107,ENSG00000251562,ENSG00000130255,ENSG00000105193,ENSG00000197061,ENSG00000137154,ENSG00000166441,ENSG00000142676,ENSG00000187514,ENSG00000083845,ENSG00000136942,ENSG00000167996,ENSG00000134419,ENSG00000205542,ENSG00000145592,ENSG00000198886,ENSG00000143947,ENSG00000125691,ENSG00000149806,ENSG00000198763,...,ENSG00000153485,ENSG00000244045,ENSG00000119718,ENSG00000177508,ENSG00000178307,ENSG00000142444,ENSG00000169180,ENSG00000100116,ENSG00000108590,ENSG00000025293,ENSG00000101266,ENSG00000072849,ENSG00000132581,ENSG00000247092,ENSG00000109111,ENSG00000102900,ENSG00000126787,ENSG00000260708,ENSG00000071564,ENSG00000153944,ENSG00000132612,ENSG00000108639,ENSG00000213015,ENSG00000167900,ENSG00000119559,ENSG00000197696,ENSG00000160209,ENSG00000105698,ENSG00000159111,ENSG00000125375,ENSG00000269858,ENSG00000182087,ENSG00000160214,ENSG00000166411,ENSG00000186153,ENSG00000089351,ENSG00000108433,ENSG00000206053,ENSG00000137806,ENSG00000197766
AATTGTGACTACGA-1,826.0,674.0,694.0,809.0,771.0,796.0,755.0,684.0,597.0,516.0,622.0,493.0,492.0,490.0,561.0,446.0,555.0,511.0,494.0,510.0,451.0,259.0,471.0,471.0,224.0,520.0,451.0,456.0,384.0,407.0,422.0,356.0,418.0,265.0,406.0,441.0,367.0,330.0,352.0,447.0,...,1.0,0.0,4.0,0.0,2.0,2.0,8.0,5.0,5.0,0.0,2.0,5.0,3.0,4.0,1.0,3.0,1.0,3.0,3.0,3.0,0.0,3.0,4.0,1.0,2.0,2.0,1.0,2.0,1.0,3.0,3.0,2.0,3.0,2.0,2.0,0.0,0.0,1.0,5.0,4.0
TGACACGATTCGTT-1,617.0,618.0,594.0,703.0,671.0,473.0,549.0,523.0,476.0,527.0,520.0,449.0,433.0,407.0,419.0,421.0,467.0,429.0,377.0,439.0,384.0,389.0,372.0,440.0,384.0,415.0,366.0,362.0,368.0,353.0,377.0,205.0,320.0,304.0,320.0,349.0,335.0,247.0,307.0,274.0,...,2.0,1.0,3.0,2.0,2.0,4.0,1.0,1.0,2.0,2.0,3.0,3.0,4.0,1.0,4.0,3.0,1.0,0.0,1.0,5.0,4.0,3.0,1.0,1.0,3.0,1.0,1.0,1.0,3.0,5.0,2.0,1.0,2.0,2.0,3.0,3.0,1.0,4.0,2.0,3.0
TGTCAGGATTGTCT-1,525.0,550.0,540.0,546.0,615.0,565.0,263.0,422.0,449.0,518.0,451.0,364.0,402.0,482.0,414.0,584.0,429.0,334.0,359.0,360.0,359.0,89.0,314.0,329.0,14.0,279.0,394.0,322.0,221.0,313.0,240.0,141.0,237.0,524.0,237.0,224.0,134.0,258.0,408.0,212.0,...,0.0,1.0,2.0,3.0,3.0,0.0,0.0,0.0,1.0,1.0,0.0,3.0,3.0,2.0,2.0,3.0,3.0,4.0,4.0,1.0,1.0,3.0,1.0,2.0,3.0,2.0,2.0,4.0,3.0,4.0,2.0,1.0,0.0,3.0,1.0,2.0,3.0,2.0,2.0,1.0
TAAGTAACGTTCTT-1,514.0,474.0,361.0,331.0,447.0,279.0,188.0,332.0,379.0,380.0,271.0,233.0,322.0,387.0,215.0,386.0,390.0,317.0,257.0,243.0,275.0,85.0,205.0,252.0,258.0,234.0,270.0,250.0,279.0,251.0,256.0,122.0,113.0,269.0,266.0,140.0,89.0,128.0,303.0,229.0,...,0.0,0.0,1.0,0.0,3.0,2.0,1.0,1.0,0.0,3.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,1.0,3.0,0.0,0.0,1.0,1.0,2.0,0.0,3.0,3.0,0.0,1.0,0.0,3.0,1.0
TCCTAAACTTCATC-1,444.0,507.0,509.0,566.0,520.0,246.0,352.0,413.0,381.0,481.0,369.0,309.0,361.0,412.0,205.0,338.0,366.0,387.0,337.0,294.0,261.0,96.0,350.0,344.0,106.0,281.0,275.0,311.0,270.0,248.0,327.0,145.0,269.0,166.0,266.0,155.0,164.0,217.0,278.0,100.0,...,2.0,0.0,0.0,0.0,2.0,3.0,2.0,3.0,3.0,1.0,1.0,2.0,2.0,0.0,0.0,3.0,0.0,3.0,1.0,2.0,4.0,1.0,2.0,3.0,1.0,3.0,1.0,3.0,0.0,2.0,1.0,2.0,0.0,5.0,4.0,0.0,3.0,4.0,2.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
AAATGGGATGCCTC-1,169.0,149.0,161.0,203.0,178.0,155.0,128.0,150.0,124.0,120.0,137.0,109.0,70.0,87.0,97.0,120.0,113.0,121.0,144.0,148.0,110.0,84.0,112.0,125.0,73.0,115.0,80.0,88.0,167.0,118.0,123.0,70.0,89.0,87.0,93.0,78.0,102.0,63.0,97.0,74.0,...,0.0,0.0,2.0,0.0,1.0,0.0,0.0,2.0,1.0,0.0,1